In [40]:
from tkinter import *
from tkinter.ttk import *
import tkinter.messagebox as messagebox
from enrich2.plugins.options import ScorerOptions, Option


class OptionFrame(Frame):
    def __init__(self, parent, options: ScorerOptions, **kw):
        super().__init__(parent, **kw)
        self.parent = parent
        self.row = 1
        self.widgets = []
        self.option_vars = []
        self.labels = []
        
        Label(self, text="Script Options: ", relief=RIDGE).grid(
            columnspan=2)
        self.row += 1
        self.parse_options(options)

        # ------------ debug ------------- #
        def get_vars():
            print(self.get_option_cfg())

        Button(master=self, text='Validate', command=get_vars).grid(
            sticky=E, column=1, row=self.row)
        # ------------ debug ------------- #
        
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=3)

    def parse_options(self, options: ScorerOptions) -> None:
        for option in options:
            try:
                option.validate(option.default)
            except TypeError:
                warn = "The default value for option {} has type" \
                       " '{}' and does not match the specified expected " \
                       "type '{}'. The program may behave unexpectedly."
                messagebox.showwarning(
                    title="Default option type does not match dtype.",
                    message=warn.format(option.name, 
                                        type(option.default).__name__, 
                                        option.dtype.__name__))
            self.create_widget_from_option(option)
            self.rowconfigure(self.row, weight=1)
            self.row += 1

    def create_widget_from_option(self, option: Option) -> None:
        if option.choices:
            self.make_choice_menu_widget(option)
        elif option.dtype in (str, 'string', 'char', 'chr'):
            self.make_string_entry_widget(option)
        elif option.dtype in ('integer', 'int', int):
            self.make_int_entry_widget(option)
        elif option.dtype in ('float', float):
            self.make_float_entry_widget(option)
        elif option.dtype in ('bool', bool, 'boolean'):
            self.make_bool_entry_widget(option)
        else:
            raise ValueError("Unrecognised attribute in option "
                             "dtype {}.".format(option.dtype))

    def make_choice_menu_widget(self, option: Option) -> None:
        menu_var = StringVar(self)
        menu_var.set(option.default)
        
        label_text = "{}: ".format(option.name)
        label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
        label.grid(sticky=EW, column=0, row=self.row)

        popup_menu = OptionMenu(
            self, menu_var, option.default, *option.choices)
        popup_menu.grid(sticky=E, column=1, row=self.row)

        self.option_vars.append((option, menu_var))
        self.widgets.append(popup_menu)
        self.labels.append(label)

    def make_entry(self, variable: Variable, option: Option) -> Entry:
        label_text = "{}: ".format(option.name)
        label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
        label.grid(sticky=EW, column=0, row=self.row)

        def validate_entry():
            try:
                value = variable.get()
                option.validate(value)
                variable.set(option.dtype(value))
            except (TclError, TypeError):
                messagebox.showwarning(
                    title="Invalid {} Entry".format(option.name),
                    message="Invalid type for entry {}. "
                            "Expected type {}.".format(option.name,
                                                       option.dtype.__name__))
                variable.set(option.dtype(option.default))
                return False
            return True

        entry = Entry(
            self, textvariable=variable,
            validate="focusout", validatecommand=validate_entry
        )
        entry.grid(sticky=EW, column=1, row=self.row)
        self.option_vars.append((option, variable))
        self.widgets.append(entry)
        self.labels.append(label)
        return entry

    def make_string_entry_widget(self, option: Option) -> None:
        variable = StringVar(self)
        variable.set(option.dtype(option.default))
        self.make_entry(variable, option)

    def make_int_entry_widget(self, option: Option) -> None:
        variable = IntVar(self)
        variable.set(option.dtype(option.default))
        self.make_entry(variable, option)

    def make_float_entry_widget(self, option: Option) -> None:
        variable = DoubleVar(self)
        variable.set(option.dtype(option.default))
        self.make_entry(variable, option)

    def make_bool_entry_widget(self, option: Option) -> None:
        variable = BooleanVar(self)
        variable.set(option.default)

        label_text = "{}: ".format(option.name)
        label = Label(self, text=label_text, justify=LEFT, relief=RIDGE)
        label.grid(sticky=EW, column=0, row=self.row)

        checkbox = Checkbutton(self, variable=variable)
        checkbox.grid(sticky=E, column=1, row=self.row)

        self.option_vars.append((option, variable))
        self.widgets.append(checkbox)
        self.labels.append(label)

    def min_frame_width(self) -> int:
        col0_max_width = max(*[l.winfo_width() for l in self.labels])
        col1_max_width = max(*[w.winfo_width() for w in self.widgets])
        return col0_max_width + col1_max_width

    def max_frame_width(self) -> int:
        col0_max_width = max(*[l.winfo_width() for l in self.labels])
        col1_max_width = max(*[w.winfo_width() for w in self.widgets])
        return (col0_max_width + col1_max_width) * 2

    def min_frame_height(self) -> int:
        col0_height = sum([l.winfo_height() for l in self.labels])
        col1_height = sum([w.winfo_height() for w in self.widgets])
        return max(*[col0_height, col1_height])

    def max_frame_height(self) -> int:
        col0_height = sum([l.winfo_height() for l in self.labels])
        col1_height = sum([w.winfo_height() for w in self.widgets])
        return max(*[col0_height, col1_height]) * 2

    def get_option_cfg(self) -> dict:
        cfg = {}
        for option, var in self.option_vars:
            value = var.get()
            option.validate(value)
            cfg[option.varname] = var.get()
        return cfg

    
options = ScorerOptions()
options.add_option(
    name="Normalization Method",
    varname="logr_method",
    dtype=str,
    default='wt',
    choices=['wt', 'full', 'complete'],
    tooltip=""
)
options.add_option(
    name="Wild-type string",
    varname="wt_string",
    dtype=str,
    default='Hello World',
    choices=[],
    tooltip=""
)
options.add_option(
    name="Alpha",
    varname="alpha",
    dtype=int,
    default=0,
    choices=[],
    tooltip=""
)
options.add_option(
    name="Beta",
    varname="beta",
    dtype=float,
    default=0.0,
    choices=[],
    tooltip=""
)
options.add_option(
    name="Weighted",
    varname="weighted",
    dtype=bool,
    default=False,
    choices=[],
    tooltip=""
)

 
root = Tk()
mainframe = OptionFrame(root, options)
mainframe.pack(side=LEFT, expand=YES, fill=BOTH, padx=5, pady=5)
root.mainloop()

{'logr_method': 'wt', 'wt_string': 'Hello World', 'alpha': 0, 'beta': 0.0, 'weighted': False}


In [ ]:
class OptionsFileFrame(Frame):

    def __init__(self, parent, options_files: ScorerOptionsFiles, **config):
        super().__init__(parent, **config)
        self.row = 1
        self.widgets = []
        self.labels = []
        self.make_buttons(options_files)

    def make_buttons(self, options_files: ScorerOptionsFiles):
        options_file: OptionsFile
        for options_file in options_files:
            pass

    def make_label_button_frame(self, options_file: OptionsFile):
        pass
    
options = ScorerOptions()
options.add_option(
    name="Beta",
    varname="beta",
    dtype=float,
    default=0.0,
    choices=[],
    tooltip=""
)
options.add_option(
    name="Weighted",
    varname="weighted",
    dtype=bool,
    default=False,
    choices=[],
    tooltip=""
)

 
root = Tk()
mainframe = OptionFrame(root, options)
mainframe.pack(side=LEFT, expand=YES, fill=BOTH, padx=5, pady=5)
root.mainloop()